In [0]:
!pip install xgboost

import pandas as pd 
import xgboost as xgb 

raw_input = pd.read_csv("/databricks-datasets/Rdatasets/data-001/csv/datasets/iris.csv",
                        header = 0,
                       names=["item","sepal length","sepal width", "petal length", "petal width","class"])
new_input = raw_input.drop(columns=["item"])
new_input["class"] = new_input["class"].astype('category')
new_input["classIndex"] = new_input["class"].cat.codes
print(new_input)

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/20/2f/2ccc91db71096cdb8c5174948db738075d7e38cfece0815500336f6e4ca8/xgboost-2.1.1-py3-none-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for nvidia-nccl-cu12 from https://files.pythonhosted.org/packages/ed/1f/6482380ec8dcec4894e7503490fc536d846b0d59694acad9cf99f27d0e7d/nvidia_nccl_cu12-2.23.4-py3-none-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/153.9 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/153.9 MB 56.2 MB/s eta 0:00:03
   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/153.9 MB 248.8 MB/s eta 0:00:01
   ━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/153.9 MB 227.9 MB/s eta 0:00:01
   ━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/153.9 MB 240.0 MB/s eta 0:00:01
   ━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.7/153.9 MB 275.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/153.9 MB 312.9 MB/s eta 0

In [0]:
from sklearn.model_selection import train_test_split
# Split to train/test
training_df, test_df = train_test_split(new_input)

In [0]:
dtrain = xgb.DMatrix(training_df[["sepal length","sepal width", "petal length", "petal width"]], label=training_df["classIndex"])

In [0]:
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'multi:softmax'}
param['nthread'] = 4
param['eval_metric'] = 'auc'
param['num_class'] = 6

In [0]:
num_round = 10
bst = xgb.train(param, dtrain, num_round)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-e1757de5-10f2-440b-9952-31453013913a/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning:

[21:19:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.




In [0]:
dtest = xgb.DMatrix(test_df[["sepal length","sepal width", "petal length", "petal width"]])
ypred = bst.predict(dtest)

In [0]:
from sklearn.metrics import precision_score

pre_score = precision_score(test_df["classIndex"],ypred, average='micro')

print("xgb_pre_score:",pre_score)

xgb_pre_score: 0.9210526315789473
